In [ ]:
import csv
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pickle
import theano
import theano.tensor as T

from sklearn import datasets
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [ ]:
with open('targets.csv') as f:
    reader = csv.reader(f)
    targets_all = next(reader)
    
print('Total '+str(len(targets_all))+' targets:')
print(targets_all)

In [ ]:
path_to_data = './data/'
year = '2020'
# targets = targets_all
# targets = ['AVG_DEMAND_KW_CALCULATED_12','NSN_LF59_BLDG-CDW-LOOP_CDW-FLOW','NSN_LF59_BLDG-CDW-LOOP_CDW-T-R','NSN_LF59_BLDG-CDW-LOOP_CDW-T-S','NSN_LF59_COOLING-TOWER_FAN-HI','NSN_LF59_COOLING-TOWER_FAN-LO','NSN_LF59_COOLING-TOWER_FAN-STAT','NSN_LF59_CWP-1-VFD_VFD-SIG','NSN_LF59_CWP-1_PMP','NSN_LF59_CWP-1_PMP-STAT','NSN_LF59_BOILER-1_HW-T-E','NSN_LF59_BOILER-1_HW-T-L','NSN_LF59_PWSHP-2_CLG-COIL-T','NSN_LF59_PWSHP-2_OA-FLOW','NSN_LF59_PWSHP-2_SA-T','NSN_LF59_PWSHP-2_SA-T-STPT','NSN_LF59_WSHP-01_DA-T','NSN_LF59_WSHP-01_LW-T','NSN_LF59_WSHP-01_ZN-STPT-CL-EFF','NSN_LF59_WSHP-01_ZN-STPT-HT-EFF','NSN_LF59_WSHP-01_ZN-T','NSN_LF59_WSHP-12_DA-T','NSN_LF59_WSHP-12_LW-T','NSN_LF59_WSHP-12_ZN-STPT-CL-EFF','NSN_LF59_WSHP-12_ZN-STPT-HT-EFF','NSN_LF59_WSHP-12_ZN-T','NSN_LF59_BuildingKWhdSum','NSN_LF59_BuildingKWHdSumAnnualized']
# targets = [targets_all[0]]
targets = ['NSN_LF59_COOLING-TOWER_FAN-LO']
for target in targets:
    print('Target: '+target)

datafilename = 'train_data_'+year+'.csv'
data = pd.read_csv(path_to_data+datafilename)

X = np.vstack((data.index.to_numpy(), data['HOD'].to_numpy(), data['Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)'].to_numpy())).T
X[:,0] = X[:,0] / 35040.0
X[:,1] = X[:,1] / 24.0
X[:,2] = (X[:,2]+20.0) / 60.0

x_train = X

In [ ]:
floatX = theano.config.floatX
RANDOM_SEED = 12356
rng = np.random.default_rng(RANDOM_SEED)
def construct_nn(X_train, Y_train):
    n_hidden = 5

    # Initialize random weights between each layer
    init_1 = rng.standard_normal(size=(X_train.shape[1], n_hidden)).astype(floatX)
    init_2 = rng.standard_normal(size=(n_hidden, n_hidden)).astype(floatX)
    init_out = rng.standard_normal(size=n_hidden).astype(floatX)

    coords = {
        "hidden_layer_1": np.arange(n_hidden),
        "hidden_layer_2": np.arange(n_hidden),
        "train_cols": np.arange(X_train.shape[1]),
        "obs_id": np.arange(X_train.shape[0]),
    }
    with pm.Model(coords=coords) as neural_network:
        # Trick: Turn inputs and outputs into shared variables using the data container pm.Data
        # It's still the same thing, but we can later change the values of the shared variable
        # (to switch in the test-data later) and pymc3 will just use the new data.
        # Kind-of like a pointer we can redirect.
        # For more info, see: http://deeplearning.net/software/theano/library/compile/shared.html
        # ann_input = pm.Data("ann_input", X_train)
        # ann_output = pm.Data("ann_output", Y_train)

        # Weights from input to hidden layer
        weights_in_1 = pm.Normal(
            "w_in_1", 0, sigma=1, testval=init_1, dims=("train_cols", "hidden_layer_1")
        )

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal(
            "w_1_2", 0, sigma=1, testval=init_2, dims=("hidden_layer_1", "hidden_layer_2")
        )

        # Weights from hidden layer to output
        weights_2_out = pm.Normal("w_2_out", 0, sigma=1, testval=init_out, dims="hidden_layer_2")

        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(X_train, weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1, weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2, weights_2_out))

        # Binary classification -> Bernoulli likelihood
        out = pm.Bernoulli(
            "out",
            act_out,
            observed=Y_train,
            total_size=Y_train.shape[0],  # IMPORTANT for minibatches
            dims="obs_id",
        )
    return neural_network

In [ ]:

for target in targets:
    print('Working on ' + target + '...')
    y_train = data[target].to_numpy()
    y_train = y_train[:, None]
    print(x_train.shape)
    print(y_train.shape)
    
    neural_network = construct_nn(x_train, y_train)
    pm.set_tt_rng(42)
    with neural_network:
        inference = pm.ADVI()
        approx = pm.fit(n=30000, method=inference)

    #with open('./model/'+target+'.pkl', 'wb') as file:
    #    pickle.dump(m, file)

    print('Model for ' + target + ' Complete!')